In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
from google.colab import files
files.upload()


Saving bengaluru_weather data.csv to bengaluru_weather data.csv
Saving ramanagar weather data.csv to ramanagar weather data.csv
Saving siddlaghatta weather data.csv to siddlaghatta weather data.csv


{'bengaluru_weather data.csv': b'YEAR,DOY,T2M,RH2M,PRECTOTCORR\r\n2021,1,18.87,81.29,0.16\r\n2021,2,19.86,84.14,0.29\r\n2021,3,20.95,82.54,0.15\r\n2021,4,21.01,80.6,0.06\r\n2021,5,20.98,86.53,0.48\r\n2021,6,21.7,86.4,13.31\r\n2021,7,21.16,87.96,4.22\r\n2021,8,22.02,85.13,0.16\r\n2021,9,22.21,83.32,0.05\r\n2021,10,21.39,82.98,0\r\n2021,11,20.86,82.66,0.01\r\n2021,12,20.05,82.01,0.09\r\n2021,13,20.66,77.67,0.12\r\n2021,14,20.41,79.63,0.22\r\n2021,15,21.23,76.72,0.04\r\n2021,16,21.45,72.58,0\r\n2021,17,21.12,74.65,0\r\n2021,18,20.16,73.55,0\r\n2021,19,20.17,74.28,0\r\n2021,20,20.87,71.24,0\r\n2021,21,22.27,71.25,0\r\n2021,22,22.55,72.91,0\r\n2021,23,21.83,71.33,0\r\n2021,24,21.05,69.06,0\r\n2021,25,19.72,66.74,0\r\n2021,26,19.64,61.53,0\r\n2021,27,19.19,65.51,0\r\n2021,28,20.04,68.92,0\r\n2021,29,21.06,69.02,0\r\n2021,30,21.74,67.06,0\r\n2021,31,21.63,62.62,0\r\n2021,32,20.36,60.89,0\r\n2021,33,20.87,66.47,0\r\n2021,34,21.49,68.2,0\r\n2021,35,21.07,63.85,0\r\n2021,36,20.61,63.04,0\r\n2021

In [ ]:
market = pd.read_csv("market_price.csv")


In [ ]:
def get_season(m):
    if m in [3,4,5]: return "Summer"
    if m in [6,7,8,9]: return "Monsoon"
    if m in [10,11]: return "PostMonsoon"
    return "Winter"

market["season"] = market["month"].apply(get_season)


In [ ]:
def load_weather(file, city):
    df = pd.read_csv(file)
    if "DOY" in df.columns:
        df["date"] = pd.to_datetime(df["YEAR"], format="%Y") + pd.to_timedelta(df["DOY"]-1, unit="D")
    else:
        df["date"] = pd.to_datetime(df["DATE"])
    df["city"] = city
    return df[["date","T2M","RH2M","PRECTOTCORR","city"]]

weather = pd.concat([
    load_weather("bengaluru_weather data.csv","Bengaluru"),
    load_weather("ramanagar weather data.csv","Ramanagar"),
    load_weather("siddlaghatta weather data.csv","Siddlaghatta")
])


In [ ]:
samples = []

for city in weather["city"].unique():
    w = weather[weather["city"]==city].sort_values("date")

    for start in w["date"]:
        end = start + timedelta(days=24)
        win = w[(w["date"]>=start)&(w["date"]<=end)]
        if len(win) < 25: continue

        harvest = end
        price = market[(market["year"]==harvest.year)&(market["month"]==harvest.month)]
        if price.empty: continue

        samples.append({
            "city": city,
            "season": get_season(start.month),
            "avg_temp": win["T2M"].mean(),
            "max_temp": win["T2M"].max(),
            "avg_humidity": win["RH2M"].mean(),
            "rainfall": win["PRECTOTCORR"].sum(),
            "price": price.iloc[0]["avg_price"]
        })


In [ ]:
data = pd.DataFrame(samples)


In [ ]:
le_city = LabelEncoder()
le_season = LabelEncoder()

data["city"] = le_city.fit_transform(data["city"])
data["season"] = le_season.fit_transform(data["season"])


In [ ]:
data.head(10)


,city,season,avg_temp,max_temp,avg_humidity,rainfall,price
0,0,2,26.6940,30.7,42.5032,0.09,446.271
1,0,2,26.8744,30.7,41.7448,0.09,446.271
2,0,2,27.1196,30.7,41.0644,0.09,446.271
3,0,2,27.2940,30.7,40.2140,0.24,446.271
4,0,2,27.4304,30.7,39.7176,0.36,446.271
5,0,2,27.5640,30.7,39.6864,0.32,446.271
6,0,2,27.7368,30.7,39.4020,0.32,446.271
7,0,2,27.8656,30.7,38.7980,0.32,446.271
8,0,2,27.9704,30.7,38.5684,0.32,446.271
9,0,2,28.0452,30.7,38.6748,0.32,446.271


In [ ]:
data.describe()


,city,season,avg_temp,max_temp,avg_humidity,rainfall,price
count,4842.000000,4842.000000,4842.000000,4842.000000,4842.000000,4842.000000,4842.000000
mean,1.000000,1.383519,24.275877,26.054097,70.553557,75.335260,604.484093
std,0.816581,1.171761,2.520831,2.764549,13.930651,76.096139,160.871059
min,0.000000,0.000000,18.510000,20.130000,36.595600,0.000000,211.744000
25%,0.000000,0.000000,22.740900,24.140000,61.373200,12.812500,499.190000
50%,1.000000,2.000000,23.960000,25.400000,75.090000,59.140000,587.599000
75%,2.000000,2.000000,25.806800,28.340000,81.243200,114.060000,742.508000
max,2.000000,3.000000,31.471600,33.260000,90.385600,597.700000,919.974000


In [ ]:
data.to_csv("training_dataset.csv", index=False)

from google.colab import files
files.download("training_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **TRAINING USING XGBOOST MODEL**


In [ ]:
!pip install xgboost


In [ ]:
from xgboost import XGBRegressor


In [ ]:
X = data.drop("price", axis=1)
y = data["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
model = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=600,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

mae, rmse, r2


(31.474902100365224, np.float64(52.24835521668643), 0.8975326068194505)

TESTING IF THE MODEL HAS LEARNED

In [ ]:
test_row = X_test.iloc[[0]]
actual_price = y_test.iloc[0]
predicted_price = model.predict(test_row)[0]

print("Actual price:", actual_price)
print("Predicted price:", predicted_price)


Actual price: 768.114
Predicted price: 704.436


In [ ]:
from datetime import timedelta

def recommend_start_date(city_name):
    city_code = le_city.transform([city_name])[0]

    city_weather = weather[weather["city"] == city_name].sort_values("date").reset_index(drop=True)

    # Use historical window so we always have 25 days available
    today = city_weather.iloc[-26]["date"]

    results = []

    for i in range(1, 21):  # test next 20 candidate start dates
        start_date = today + pd.Timedelta(days=i)
        end_date = start_date + pd.Timedelta(days=24)

        win = city_weather[(city_weather["date"] >= start_date) & (city_weather["date"] <= end_date)]

        if len(win) < 25:
            continue

        features = pd.DataFrame([{
            "city": city_code,
            "season": le_season.transform([get_season(start_date.month)])[0],
            "avg_temp": win["T2M"].mean(),
            "max_temp": win["T2M"].max(),
            "avg_humidity": win["RH2M"].mean(),
            "rainfall": win["PRECTOTCORR"].sum()
        }])

        predicted_price = model.predict(features)[0]
        results.append((start_date.date(), predicted_price))

    results = sorted(results, key=lambda x: -x[1])
    return results


In [ ]:
recommendations = recommend_start_date("Bengaluru")

for d, p in recommendations[:5]:
    print("Start Date:", d, "→ Expected Price: ₹", round(p, 2))


Start Date: 2025-12-02 → Expected Price: ₹ 424.46


MODEL VALIDATION CELLS

In [ ]:
sample = X_test.sample(10)
actual = y_test.loc[sample.index]
predicted = model.predict(sample)

comparison = pd.DataFrame({
    "Actual": actual.values,
    "Predicted": predicted
})

comparison


,Actual,Predicted
0,577.483,561.776245
1,617.313,624.858765
2,554.346,568.422607
3,619.326,639.534973
4,874.536,839.325867
5,516.996,578.360535
6,549.951,616.959595
7,663.184,608.076782
8,549.951,546.345886
9,577.350,582.579163


In [ ]:
pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))
r2 = r2_score(y_test, pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)


MAE: 31.474902100365224
RMSE: 52.24835521668643
R²: 0.8975326068194505
